In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [72]:
# API Call
load_dotenv()
API_KEY = os.getenv("Yelp_API") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"

In [73]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204


In [74]:
# Make a list of coordinate tuples (lat, lng, city) for each team
stadium_zip = zip(stadium_data_df["Latitude"], stadium_data_df["Longitude"], stadium_data_df['NFL Team'])
stadium = list(stadium_zip)
#print(stadium))

In [75]:
# API Parameters
search_limit = 50 #limit to 50 business names per query
radius = 3000 #radius in meters
term = "restaurants"
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating"}

# lists to store the data retrieve from Yelp
business_name = []
business_state = []
#business_lat = []
#business_lng = []
business_price = []
business_rating = []
business_reviews = []
number_business = []
print(stadium)

[(33.528, -112.26299999999999, 'Arizona Cardinals'), (33.755556, -84.4, 'Atlanta Falcons'), (39.278056, -76.622778, 'Baltimore Ravens'), (42.773999999999994, -78.78699999999999, 'Buffalo Bills'), (35.225833, -80.852778, 'Carolina Panthers'), (41.8623, -87.6167, 'Chicago Bears'), (39.095, -84.516, 'Cincinnati Bengals'), (41.506111, -81.699444, 'Cleveland Browns'), (32.747778000000004, -97.09277800000001, 'Dallas Cowboys'), (39.741, -105.0197, 'Denver Broncos'), (42.34, -83.045556, 'Detroit Lions'), (44.501389, -88.06222199999999, 'Green Bay Packers'), (29.684721999999997, -95.410833, 'Houston Texans'), (39.760056, -86.16380600000001, 'Indianapolis Colts'), (30.323889, -81.6375, 'Jacksonville Jaguars'), (39.048889, -94.483889, 'Kansas City Chiefs'), (36.090833, -115.18361100000001, 'Las Vegas Raiders'), (33.95345, -118.3392, 'Los Angeles Chargers'), (33.95345, -118.3392, 'Los Angeles Rams'), (25.958056, -80.238889, 'Miami Dolphins '), (44.974, -93.258, 'Minnesota Vikings'), (42.091, -71.

In [76]:
#Loop to dump data into json files based on (lat,lng)
count_f = 0
for latitude, longitude, team in stadium:
        
    search_params2.update({"latitude": latitude, "longitude": longitude})
    response_test = requests.get(url = base_url, params = search_params2, headers = headers).json()
    ntotal = response_test['total']
        
    with open(f'json_files\Saturday\yelp_response_{team}.json','w', encoding = 'utf-8') as f:
        json.dump(response_test, f, ensure_ascii=False, indent =4)
    f.close()
    print(f'{count_f}, {city} total: {ntotal}')
    
    count_f += 1
        

0, Landover total: 94
1, Landover total: 706
2, Landover total: 667
3, Landover total: 51
4, Landover total: 440
5, Landover total: 1400
6, Landover total: 458
7, Landover total: 354
8, Landover total: 210
9, Landover total: 575
10, Landover total: 530
11, Landover total: 120
12, Landover total: 180
13, Landover total: 354
14, Landover total: 175
15, Landover total: 18
16, Landover total: 758
17, Landover total: 293
18, Landover total: 293
19, Landover total: 84
20, Landover total: 682
21, Landover total: 44
22, Landover total: 946
23, Landover total: 133
24, Landover total: 133
25, Landover total: 396
26, Landover total: 380
27, Landover total: 179
28, Landover total: 1400
29, Landover total: 270
30, Landover total: 528
31, Landover total: 94


#read json files
counter = 0
file_list = os.listdir('json_files\Saturday') 
for file in file_list:
    
    with open(f'json_files\Saturday\{file}', 'r', encoding = 'utf-8') as f:
        contents = json.loads(f.read())
        
    print(counter, contents['total'], file)
    counter += 1